In [1]:
from botasaurus.browser import Driver, Wait, browser
from botasaurus.user_agent import UserAgent
from botasaurus.window_size import WindowSize
import time
import subprocess

# Informasi login dan akses WordPress
WP_URL = "http://portal-berita.local/wp-login.php?loggedout=true&wp_lang=en_US"
USERNAME = "admin"
PASSWORD = "admin"
NEW_POST_URL = "https://your-wordpress-site.com/wp-admin/post-new.php"

d = None
@browser(
    reuse_driver=True,
    close_on_crash=True,
    user_agent=UserAgent.HASHED,
    window_size=WindowSize.HASHED,
    output=None,
)
def login_wordpress(driver: Driver, data):
    """Login ke dashboard WordPress."""
    global d
    driver.google_get(WP_URL, accept_google_cookies=True, wait=Wait.SHORT)

    # Isi username dan password
    driver.type("input[id='user_login']", USERNAME, Wait.SHORT)
    driver.type("input[id='user_pass']", PASSWORD, Wait.SHORT)

    # Klik tombol login
    driver.click("input[id='wp-submit']", Wait.SHORT)
    time.sleep(2)

    print("[*] Login berhasil")
    # driver.google_get(NEW_POST_URL, wait=Wait.LONG)
    # style_content(driver)
    upload_page(driver)
    d = driver

def wait_for_iframe(driver: Driver, iframe_selector: str, timeout=10):
    """Menunggu iframe tersedia dan siap digunakan."""
    start_time = time.time()
    while time.time() - start_time < timeout:
        iframes = driver.select_all(iframe_selector, wait=Wait.SHORT)
        if iframes:
            return iframes[0]  # Kembalikan iframe pertama yang ditemukan
        time.sleep(1)
    raise Exception(f"Iframe '{iframe_selector}' tidak ditemukan dalam batas waktu.")

def upload_page(driver: Driver):
    # Buka halaman untuk menambahkan page baru
    driver.google_get(
        "http://portal-berita.local/wp-admin/edit.php?post_type=page",
        bypass_cloudflare=False,
        wait=Wait.SHORT,
    )

    # Klik tombol "Add New Page"
    add_new_page = driver.select(".page-title-action", wait=Wait.SHORT).get_attribute(
        "href"
    )
    driver.google_get(add_new_page, bypass_cloudflare=False, wait=Wait.SHORT)

    # Tutup popup jika ada
    close_popup_script = """
        const closeBtn = document.querySelector('.components-modal__header .components-button.has-icon');
        if (closeBtn) closeBtn.click();
    """
    time.sleep(5)
    driver.run_js(close_popup_script)
    # driver.reload()
    time.sleep(2)
    return driver
    # # Masukkan konten baru ke dalam editor
    # insert_content_script = """
    #     const editor = document.querySelector('.block-editor-rich-text__editable.block-editor-block-list__block.wp-block.is-selected.wp-block-paragraph.rich-text');
    #     console.log(editor)
    #     editor.focus();
    #     editor.innerHTML = "<p>Halo, ini konten baru</p>";

    #     // Trigger event 'input' agar WordPress mendeteksi perubahan
    #     const event = new Event('input', { bubbles: true });
    #     editor.dispatchEvent(event);
    # """
    # result = driver.run_js(insert_content_script)
    # print(result)
    # r = driver.select(
    #     selector=".block-editor-rich-text__editable.block-editor-block-list__block.wp-block.is-selected.wp-block-paragraph.rich-text",
    #     wait=Wait.SHORT,
    # )
    # time.sleep(5)
    # print(r)

    # # Kembali ke konteks utama
    # # driver.switch_to_default_content()

    # # Klik tombol "Publish"
    # publish_script = """
    #     const publishBtn = document.querySelector('.editor-post-publish-button');
    #     if (publishBtn) {
    #         publishBtn.click();
    #         return "Page berhasil dipublikasikan.";
    #     } else {
    #         return "Tombol Publish tidak ditemukan.";
    #     }
    # """
    # result = driver.run_js(publish_script)
    # print(result)

    # time.sleep(5)  # Jeda agar publikasi selesai

    # # Kembali ke konteks utama jika menggunakan iframe
    # driver.switch_to_default_content()

    # # Klik tombol "Publish"
    # publish_script = """
    #     const publishBtn = document.querySelector('.editor-post-publish-button');
    #     if (publishBtn) {
    #         publishBtn.click();
    #         return "Page berhasil dipublikasikan.";
    #     } else {
    #         return "Tombol Publish tidak ditemukan.";
    #     }
    # """
    # result = driver.run_js(publish_script)
    # print(result)

    # time.sleep(5)  # Jeda untuk memastikan publikasi selesai


def style_content(driver: Driver):
    """Mengatur konten dengan drag-and-drop dan styling block."""
    time.sleep(2)  # Beri jeda agar halaman sepenuhnya dimuat

    # Simulasi menambah block baru dengan klik
    driver.click("button[aria-label='Add block']", Wait.SHORT)

    # Pilih block 'Paragraph' menggunakan JavaScript injection
    js_code = """
    const blocks = document.querySelectorAll('button.editor-block-list-item-paragraph');
    if (blocks.length > 0) {
        blocks[0].click();
    }
    """
    driver.execute_script(js_code)
    time.sleep(1)

    # Isi teks paragraf
    paragraph_js = """
    const paragraph = document.querySelector('.block-editor-rich-text__editable');
    if (paragraph) {
        paragraph.innerHTML = '<p>Ini adalah paragraf baru dengan drag-and-drop otomatis!</p>';
    }
    """
    driver.execute_script(paragraph_js)

    print("[*] Paragraf berhasil ditambahkan")

    # Simulasi styling block menggunakan JavaScript
    style_js = """
    const toolbarButton = document.querySelector('button[aria-label="Change alignment"]');
    if (toolbarButton) {
        toolbarButton.click();
        document.querySelector('button[aria-label="Align center"]').click();
    }
    """
    driver.execute_script(style_js)
    print("[*] Paragraf diatur menjadi align-center")

    # Publikasikan postingan
    publish_post(driver)


def publish_post(driver: Driver):
    """Publikasikan postingan."""
    driver.click("button.editor-post-publish-panel__toggle", Wait.SHORT)
    time.sleep(1)
    driver.click("button.editor-post-publish-button", Wait.SHORT)

    print("[*] Konten berhasil dipublikasikan")


# Jalankan automasi login dan styling konten
login_wordpress()

Running
Sleeping for 4 seconds...
[*] Login berhasil
Sleeping for 4 seconds...
Sleeping for 4 seconds...


In [6]:
# langsung membuka editor new post
d.google_get("http://portal-berita.local/wp-admin/post-new.php")


<Tab [FFED12D280BD33331CC1B2AEB35A2078] [page] [url: http://portal-berita.local/wp-admin/post-new.php]>

In [2]:
# menulis title kemudian post
cmd_js_insert_title = """
const iframe = document.querySelector("iframe[name='editor-canvas']").contentDocument.documentElement

// Ambil elemen h1 yang dapat diedit
const editableTitle = iframe.querySelector('.editor-post-title__input');

// Fungsi untuk mengupdate teks judul dan memicu event perubahan
function updateTitle(newTitle) {
    editableTitle.innerText = newTitle; // Mengubah teks di dalam elemen

    // Memicu event 'input' agar WordPress mendeteksi perubahan
    const event = new Event('input', { bubbles: true });
    editableTitle.dispatchEvent(event);
}

// Mengatur judul baru langsung tanpa prompt
const newTitle = "Berita terkini"; // Ganti dengan judul yang diinginkan
updateTitle(newTitle);

"""
d.run_js(cmd_js_insert_title)

In [7]:

cmd_open_template = """
    //https://www.w3schools.com/howto/howto_js_element_iframe.asp
    const iframe = document.querySelector("iframe[id='elementor-preview-iframe']").contentDocument.documentElement
    const elementorAddBtns = iframe.getElementsByClassName("e-view elementor-add-new-section")[0]
    elementorAddBtns.getElementsByClassName("elementor-add-section-area-button")[1].click()
    document.getElementsByClassName("elementor-component-tab")[4].click();
"""

cmd_popup_select_template = """
    const templates = document.querySelectorAll("div#elementor-template-library-templates-container > div");
    const totalTemplate = templates.length;
    console.log(totalTemplate)
    console.log(templates[totalTemplate -1])

    // Mencari template yg terbaru
    const insertBtn = templates[totalTemplate - 1].getElementsByClassName('elementor-template-library-template-action');
    insertBtn[0].click();

    // Tunggu hingga popup notifikasi verifikasi muncul
    const btnNotifikasi = document.getElementById("elementor-insert-template-settings-dialog");
    btnNotifikasi.getElementsByClassName("dialog-confirm-ok")[0].click();
"""

d.run_js(cmd_open_template)
d.run_js(cmd_popup_select_template)



# print(d.page_html)

In [97]:
d.run_js("""


// Fungsi sleep yang mengembalikan Promise
function sleep(ms) {
    return new Promise(resolve => setTimeout(resolve, ms));
}

(async () => {
    // Klik my template
    await waitForElement(".elementor-component-tab"); // Tunggu hingga tab muncul
    document.getElementsByClassName("elementor-component-tab")[4].click();

    // Tunggu hingga semua template muncul
    await waitForElement("div#elementor-template-library-templates-container > div");

    // Mencari semua template
    const templates = document.querySelectorAll("div#elementor-template-library-templates-container > div");
    const totalTemplate = templates.length;

    // Tunggu hingga tombol insert muncul
    await waitForElement('.elementor-template-library-template-action');
    const insertBtn = templates[totalTemplate - 1].getElementsByClassName('elementor-template-library-template-action');
    insertBtn[0].click();

    // Tunggu hingga popup notifikasi verifikasi muncul
    await waitForElement("#elementor-insert-template-settings-dialog");
    const btnNotifikasi = document.getElementById("elementor-insert-template-settings-dialog");
    btnNotifikasi.getElementsByClassName("dialog-confirm-ok")[0].click();
})();



    //document.getElementById("elementor-template-library-header-import").getElementsByTagName("i")[0].click()
    //const fileInput = document.querySelector('input[id="elementor-template-library-import-form-label"]');
""")

ElementWithSelectorNotFoundException: Cannot find element with selector: '.elementor-add-section-area-button:nth-child(2)'.

# d.google_get("http://portal-berita.local/wp-admin/edit.php?post_type=elementor_library&tabs_group=library")
# d.run_js("""
# //document.getElementById('elementor-import-template-trigger').click()

# document.getElementById('e-import-template-action').addEventListener('click', function() {
#     const fileInput = document.querySelector("input[type='file']");
#     const file = "/home/biru/Project/automaation-beritaa/template.json"; // Ambil file yang dipilih






# """)






In [46]:
d.click("input[type='file']")

In [49]:
from pynput.keyboard import Key, Controller

#keyboad.release(Key.enter)

keyboad.type("/home/biru/Project/automaation-beritaa/template.json")